This utilizes Aswath Damodaran's 2018 blog post, [Is there a signal in the noise? Yield Curves, Economic Growth and Stock Prices!](https://aswathdamodaran.blogspot.com/2018/12/is-there-signal-in-noise-yield-curves.html)

### Libraries

In [1]:
import pandas as pd
import keras
from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense, Lambda
from keras.layers import Flatten, LSTM
from keras.layers import GlobalMaxPooling1D
from keras.models import Model
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.layers import Input
from keras.layers.merge import Concatenate
from keras.layers import Bidirectional
from keras import backend as K
import tensorflow as tf
from keras.layers import RepeatVector, TimeDistributed
import numpy as np
import shap

### Data Preprocessing

In [2]:
X = pd.read_csv('./data/aswathtestinginput.csv')
Y = pd.read_csv('./data/aswathtestingoutput.csv')    
X['QuarterEnd']= pd.to_datetime(X['QuarterEnd'])
X = X.set_index(['QuarterEnd'])
X.astype({col: np.float64 for col in X.columns[1:]})
Y['QuarterEnd'] = pd.to_datetime(Y['QuarterEnd'])
Y = Y.set_index(['QuarterEnd'])
Y.astype({col: np.float64 for col in Y.columns[1:]})

,DuringQuarter,NextQuarter,NextYear
QuarterEnd,,,
1962-04-01,0.0757,0.0673,0.0516
1962-07-01,0.0673,0.0600,0.0443
1962-10-01,0.0600,0.0431,0.0414
1963-01-01,0.0431,0.0360,0.0435
1963-04-01,0.0360,0.0382,0.0500
...,...,...,...
2017-01-01,0.0188,0.0194,0.0222
2017-04-01,0.0194,0.0211,0.0238
2017-07-01,0.0211,0.0234,0.0257


In [3]:
X

,3-monthT.Bill,1-yearT.Bill,2-yearT.Bond,5-yearT.Bond,10-yearT.Bond,BaaBondRate,FedFundsRate,3mthvsFedFunds,1yrvs3mth,2yrvs1yr,5yrvs2yr,10yrvs2yr,10yrvs3mth,BaaSpread,StockReturnduringquarter,GDPGrowth
QuarterEnd,,,,,,,,,,,,,,,,
1962-04-01,0.0272,0.0297,0.0329,0.0361,0.0386,0.0504,0.0278,-0.0232,0.0025,0.0032,0.0032,0.0057,0.0114,0.0118,-0.0131,0.0757
1962-07-01,0.0273,0.0320,0.0348,0.0376,0.0400,0.0502,0.0271,-0.0229,0.0047,0.0028,0.0028,0.0052,0.0127,0.0102,-0.2012,0.0673
1962-10-01,0.0278,0.0302,0.0334,0.0366,0.0394,0.0503,0.0290,-0.0225,0.0024,0.0032,0.0032,0.0060,0.0116,0.0109,0.0520,0.0600
1963-01-01,0.0287,0.0305,0.0331,0.0356,0.0385,0.0492,0.0292,-0.0205,0.0018,0.0026,0.0025,0.0054,0.0098,0.0107,0.0892,0.0431
1963-04-01,0.0289,0.0309,0.0340,0.0371,0.0395,0.0488,0.0290,-0.0199,0.0020,0.0031,0.0031,0.0055,0.0106,0.0093,0.0570,0.0360
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-01-01,0.0051,0.0085,0.0120,0.0193,0.0245,0.0483,0.0065,-0.0432,0.0034,0.0035,0.0073,0.0125,0.0194,0.0238,0.0465,0.0188
2017-04-01,0.0074,0.0103,0.0127,0.0193,0.0240,0.0468,0.0090,-0.0394,0.0029,0.0024,0.0066,0.0113,0.0166,0.0228,0.0587,0.0194
2017-07-01,0.0098,0.0124,0.0138,0.0189,0.0231,0.0437,0.0115,-0.0339,0.0026,0.0014,0.0051,0.0093,0.0133,0.0206,0.0334,0.0211


In [4]:
X.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 224 entries, 1962-04-01 to 2018-01-01
Data columns (total 16 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   3-monthT.Bill             224 non-null    float64
 1   1-yearT.Bill              224 non-null    float64
 2   2-yearT.Bond              224 non-null    float64
 3   5-yearT.Bond              224 non-null    float64
 4   10-yearT.Bond             224 non-null    float64
 5   BaaBondRate               224 non-null    float64
 6   FedFundsRate              224 non-null    float64
 7   3mthvsFedFunds            224 non-null    float64
 8   1yrvs3mth                 224 non-null    float64
 9   2yrvs1yr                  224 non-null    float64
 10  5yrvs2yr                  224 non-null    float64
 11  10yrvs2yr                 224 non-null    float64
 12  10yrvs3mth                224 non-null    float64
 13  BaaSpread                 224 non-null    floa

In [5]:
def timestep_setup(X,timestep):
    X = X.to_numpy()
    X = array(X).reshape(int(224/timestep), timestep, 16)
    return X

### Many to One LSTM Models with Multiple Features

The X dataframe shows 16 features over 224 quarters.

In [6]:
def simple_lstm(X,Y,viz,test_input):
    model = Sequential()
    model.add(LSTM(50, activation='relu', input_shape=(4,16)))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    history = model.fit(X, Y, epochs=500, validation_split=0.2, verbose=0)
    model.summary()
    test_output = model.predict(test_input, verbose=0)
    print(test_output)
  #  if viz == 'shap':
  #      shap_viz(model,X,test_input)

In [7]:
def stacked_lstm(X,Y,test_input):
    model = Sequential()
    model.add(LSTM(200, activation='relu', return_sequences=True, input_shape=(4,16)))
    model.add(LSTM(100, activation='relu', return_sequences=True))
    model.add(LSTM(50, activation='relu', return_sequences=True))
    model.add(LSTM(25, activation='relu'))
    model.add(Dense(20, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    model.summary()
    history = model.fit(X, Y, epochs=500, validation_split=0.2, verbose=0)
    test_output = model.predict(test_input, verbose=0)
    print(test_output)

In [8]:
def bidirectional_lstm(X,Y,test_input):
    model = Sequential()
    model.add(Bidirectional(LSTM(50, activation='relu'), input_shape=(4,16)))
    model.add(Dense(2))
    model.compile(optimizer='adam', loss='mse')
    model.summary()
    history = model.fit(X, Y, epochs=500, validation_split=0.2, verbose=0)
    test_output = model.predict(test_input, verbose=0)
    print(test_output)

### Visualizations

In [9]:
def shap_viz(model,X_train,X_test):
    
    explainer = shap.DeepExplainer(model, X_train)
    shap_value = explainer.shap_values(X_test)
    shap_val = np.array(shap_value)
    a = np.absolute(shap_val[0])
    b = np.sum(a, axis=1)
    SHAP_list = [np.sum(b[:, 0]), np.sum(b[:, 1]), np.sum(b[:, 2]), np.sum(b[:, 3]), np.sum(b[:, 4])]
    N_weight = normalize(weight_list)
    N_SHAP = normalize(SHAP_list)

### Executions

In [10]:
testinput1q = np.array([0.017,0.0209,0.0227,0.0256,0.0274,0.0464,0.0169,-0.0339,0.0039,0.0018,0.0029,0.0047,0.0018,0.019,0.0191,0.0258])
test_input_nextquarter = array(testinput1q).reshape(1, 1, 16)

test_output_nextquarter = [0.0287]

In [11]:
simple_lstm(timestep_setup(X,4),Y['NextQuarter'].to_numpy(),'shap',test_input_nextquarter)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 50)                13400     
_________________________________________________________________
dense (Dense)                (None, 1)                 51        
Total params: 13,451
Trainable params: 13,451
Non-trainable params: 0
_________________________________________________________________
[[0.00607706]]


In [12]:
stacked_lstm(timestep_setup(X,4),Y['NextQuarter'].to_numpy(),test_input_nextquarter)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 4, 200)            173600    
_________________________________________________________________
lstm_2 (LSTM)                (None, 4, 100)            120400    
_________________________________________________________________
lstm_3 (LSTM)                (None, 4, 50)             30200     
_________________________________________________________________
lstm_4 (LSTM)                (None, 25)                7600      
_________________________________________________________________
dense_1 (Dense)              (None, 20)                520       
_________________________________________________________________
dense_2 (Dense)              (None, 10)                210       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                

In [13]:
bidirectional_lstm(timestep_setup(X,4),Y['NextQuarter'].to_numpy(),test_input_nextquarter)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional (Bidirectional (None, 100)               26800     
_________________________________________________________________
dense_4 (Dense)              (None, 2)                 202       
Total params: 27,002
Trainable params: 27,002
Non-trainable params: 0
_________________________________________________________________
[[0.01369886 0.01773703]]
